In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
set_session(session)

Using TensorFlow backend.


In [2]:
from keras.models import load_model

model = load_model("gz_dev.model")
print(model.summary())

OSError: Unable to open file (unable to open file: name = 'gz_dev.model.old', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import random

from trainingHelper import TrainingHelper
from trainingGame import loadTrainingGame
import nnHelper
import numpy as np

from os import listdir
from os.path import isfile, join

filepath = "D:/GalconZero/Games/"

BATCH_SIZE = 64

def getTrainingDataForNumGames(num):
    onlyFiles = [f for f in listdir(filepath) if isfile(join(filepath, f)) and ".pickle" in f]
 
    trainX = np.empty((0, 22, 14))
    trainY = [np.empty((0, 925)), np.empty((0,))]

    for filename in random.sample(onlyFiles, num):
    #     print("loading training file {}".format(filename))
        trainingGame = loadTrainingGame(filepath + "/" + filename)
        helper = TrainingHelper(trainingGame)    
        
        nextTrainX = helper.getTrainX()
        #twice, once for regular, once for flipped Y values
        trainX = np.concatenate((trainX, nextTrainX))
        trainX = np.concatenate((trainX, getXAxisReflectedTrainingData(nextTrainX)))
        
        [policy, value] = helper.getTrainY()
        #twice, once for regular, once for flipped Y values (reflecting over x axis does not change policy or value)
        trainY = [np.concatenate((trainY[0], policy)), np.concatenate((trainY[1], value))]
        trainY = [np.concatenate((trainY[0], policy)), np.concatenate((trainY[1], value))]

    print("Finished loading {} training files".format(num))
    return trainX, trainY

def trainingLoop():
    trainX, [trainYPolicy, trainYValue] = getTrainingDataForNumGames(60)
    
    indices = np.arange(len(trainX))
    randomSubsetIndices = np.random.choice(indices, size=512, replace=False)
    
    trainX = np.take(trainX, randomSubsetIndices, axis=0)
    trainYPolicy = np.take(trainYPolicy, randomSubsetIndices, axis=0)
    trainYValue = np.take(trainYValue, randomSubsetIndices, axis=0)
    
    return model.fit(trainX, [trainYPolicy, trainYValue], validation_split=0.15, epochs=5, shuffle=True, batch_size=BATCH_SIZE)

def getXAxisReflectedTrainingData(data):
    data = data.copy()
    for sample in data:
        for row in sample:
            row[7] *= -1
            row[10] *= -1
            row[13] *= -1
    return data


In [ ]:
historyPolicyValAcc = []
historyValueValAcc = []

historyPolicyValLoss = []
historyValueValLoss = []
historyValLoss = []

def doTrain(num_loops):
    for i in range(num_loops):
        history = trainingLoop()

        historyPolicyValAcc.extend(history.history['val_policy_acc'])
        historyValueValAcc.extend(history.history['val_value_acc'])
        
        historyPolicyValLoss.extend(history.history['val_policy_loss'])
        historyValueValLoss.extend(history.history['val_value_loss'])
        historyValLoss.extend(history.history['val_loss'])


In [ ]:
doTrain(5)

In [ ]:
import matplotlib.pyplot as plt 
    
plt.figure(1, figsize=(18, 18))  
   
# summarize history for accuracy  
   
plt.subplot(211)  
# plt.plot(history.history['policy_acc'])  
plt.plot(historyPolicyValAcc)  
# plt.plot(history.history['value_acc'])  
plt.plot(historyValueValAcc)  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
# plt.legend(['policy_categorical_accuracy', 'val_policy_categorical_accuracy', 'value_categorical_accuracy', 'val_value_categorical_accuracy'], loc='upper left')  
plt.legend(['Validation Policy Accuracy','Validation Value Accuracy'], loc='upper left')  
   
# summarize history for loss  
   
plt.subplot(212)  
# plt.plot(history.history['policy_loss'])  
plt.plot(historyPolicyValLoss)  
# plt.plot(history.history['value_loss'])  
plt.plot(historyValueValLoss)  
# plt.plot(history.history['loss'])  
plt.plot(historyValLoss)  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
# plt.legend(['policy_loss', 'val_policy_loss', 'value_loss','val_value_loss','loss','val_loss',], loc='upper left')  
plt.legend(['val_policy_loss','val_value_loss','val_loss'], loc='upper left')  

plt.show()  

In [ ]:
model.save("gz_dev.model")